# Location
Test notebook to make predictions at specific locations

In [ ]:
using NetCDF

In [ ]:
lons = collect(linspace(0.125,359.875,1440));
lats = collect(linspace(-89.875, 89.875, 720));

In [ ]:
lon_idx = 881+35;
lat_idx = 551+17;

In [ ]:
year = 2006

tasmax_f = string("data/test/tasmax_day_BCSD_rcp45_r1i1p1_BNU-ESM_", year, ".nc")
tasmin_f = string("data/test/tasmin_day_BCSD_rcp45_r1i1p1_BNU-ESM_", year, ".nc")

nc_tasmax = NetCDF.open(tasmax_f);
nc_tasmin = NetCDF.open(tasmin_f);


ts_max = NetCDF.readvar(nc_tasmax, "tasmax", start=[lon_idx,lat_idx,1], count=[1,1,-1]);
ts_min = NetCDF.readvar(nc_tasmin, "tasmin", start=[lon_idx,lat_idx,1], count=[1,1,-1]);

ts_max1 = vcat(ts_max...);
ts_min1 = vcat(ts_min...);

grow_range = (ts_min1 .> 273.15) .& (ts_max1 .< (273.15+34))
gdd1 = ((ts_max1 + ts_min1)/2 - 273.15) .* grow_range;

In [ ]:
year = 2007

tasmax_f = string("data/test/tasmax_day_BCSD_rcp45_r1i1p1_BNU-ESM_", year, ".nc")
tasmin_f = string("data/test/tasmin_day_BCSD_rcp45_r1i1p1_BNU-ESM_", year, ".nc")

nc_tasmax = NetCDF.open(tasmax_f);
nc_tasmin = NetCDF.open(tasmin_f);


ts_max = NetCDF.readvar(nc_tasmax, "tasmax", start=[lon_idx,lat_idx,1], count=[1,1,-1]);
ts_min = NetCDF.readvar(nc_tasmin, "tasmin", start=[lon_idx,lat_idx,1], count=[1,1,-1]);

ts_max2 = vcat(ts_max...);
ts_min2 = vcat(ts_min...);

grow_range = (ts_min2 .> 273.15) .& (ts_max2 .< (273.15+34))
gdd2 = ((ts_max2 + ts_min2)/2 - 273.15) .* grow_range;

In [ ]:
gdds = vcat([gdd1, gdd2]...);

In [ ]:
agdd = 1665;

In [ ]:
grow_range = gdds.>0
grow_prob = [sum(cumprod(grow_range[i:(i+364)]) .* gdds[i:(i+364)], 1)[1] > agdd for i in 1:(size(gdds,1)-365)];

In [ ]:
grow_count = Array{Int64,1}(0)

for i in 1:5
   
    if size(grow_prob) != size(grow_count)
        grow_count = grow_prob
    else
        grow_count += grow_prob
        
    end
    
end

## OLDER

In [ ]:
using NetCDF

In [ ]:
lon_idx = 881
lat_idx = 551

In [ ]:
year = 2006;
tasmax_f = string("data/test/tasmax_day_BCSD_rcp45_r1i1p1_BNU-ESM_", year, ".nc")
tasmin_f = string("data/test/tasmin_day_BCSD_rcp45_r1i1p1_BNU-ESM_", year, ".nc")
nc_tasmax = NetCDF.open(tasmax_f);
nc_tasmin = NetCDF.open(tasmin_f);
lons = NetCDF.readvar(nc_tasmax, "lon");
lats = NetCDF.readvar(nc_tasmax, "lat");

In [ ]:
ts_max = NetCDF.readvar(nc_tasmax, "tasmax", start=[lon_idx,lat_idx,1], count=[1,1,-1]);
ts_min = NetCDF.readvar(nc_tasmin, "tasmin", start=[lon_idx,lat_idx,1], count=[1,1,-1]);
ts_max = vcat(ts_max...);
ts_min = vcat(ts_min...);

In [ ]:
NetCDF.close(nc_tasmax);
NetCDF.close(nc_tasmin);

# DAILY MAP

In [ ]:
using NetCDF

In [ ]:
ttop = 34 + 273.15;
tbase = 5 + 273.15;

In [ ]:
tasmax = ncread(string("data/test/tasmax_day_BCSD_rcp45_r1i1p1_BNU-ESM_", 2006, ".nc"), "tasmax");
tasmin = ncread(string("data/test/tasmin_day_BCSD_rcp45_r1i1p1_BNU-ESM_", 2006, ".nc"), "tasmin");
    

In [ ]:
@time grow_range = (tasmin.>tbase) .& (tasmax.<ttop);

In [ ]:
gdd = ((tasmax + tasmin) / 2 - tbase) .* grow_range

In [ ]:
# calc_gdd

function calc_gdd(cmip::String, rcp::String, year::Int64, ttop::Float64, tbase::Float64, data_dir::String)

    println(string("\t\tCalculating gdd for year ", year))
    
    # Test
    tasmax = ncread(string("data/test/tasmax_day_BCSD_rcp45_r1i1p1_BNU-ESM_", year, ".nc"), "tasmax")
    tasmin = ncread(string("data/test/tasmin_day_BCSD_rcp45_r1i1p1_BNU-ESM_", year, ".nc"), "tasmin")
    
    
    
    """
    tasmax_f = string(data_dir, "BCSD/", rcp, "/day/atmos/tasmax/r1i1p1/v1.0/tasmax_day_BCSD_", rcp, "_r1i1p1_", cmip, "_", year, ".nc")
    tasmin_f = string(data_dir, "BCSD/", rcp, "/day/atmos/tasmin/r1i1p1/v1.0/tasmin_day_BCSD_", rcp, "_r1i1p1_", cmip, "_", year, ".nc")

    println("\t\tReading tasmax")
    nc_tasmax = NetCDF.open(tasmax_f)
    tasmax = NetCDF.readvar(nc_tasmax, "tasmax")
    NetCDF.close(nc_tasmax)
    #tasmax = ncread(tasmax_f, "tasmax")

    println("\t\tReading tasmin")
    nc_tasmin = NetCDF.open(tasmin_f)
    tasmin = NetCDF.readvar(nc_tasmin, "tasmin")
    NetCDF.close(nc_tasmin)
    #tasmin = ncread(tasmin_f, "tasmin")

    println("\t\tCalculating gdd")
    (tasmax + tasmin) / 2 - 273.15 - tbase
    """
end

# OLD

# convert map results to matrix

In [ ]:
@everywhere function calc_agdd_growth(gdd_ts::Array{Float64,1}, ttop::Float64, tbase::Float64, agdd::Float64)
   
    grow_range = (gdd_ts.>(0)) .& (gdd_ts.<(ttop-tbase))
    [sum(cumprod(grow_range[i:(i+364)]) .* gdd_ts[i:(i+364)], 1)[1] > agdd for i in 1:365]

end

In [ ]:
@time res = calc_agdd_growth(rand(730), 0.75, 0.25, 0.01);

In [ ]:
x1 = rand(10,10,730)-0.25;

In [ ]:
@time res1 = pmap(f -> calc_agdd_growth(f, 0.75, 0.25, 0.2), [x1[i,t,:] for i in 1:size(x1)[1], t in 1:size(x1)[2]]);

In [ ]:
x2 = rand(10,10,730)-0.25;

In [ ]:
@time res2 = pmap(f -> calc_agdd_growth(f, 0.75, 0.25, 0.2), [x2[i,t,:] for i in 1:size(x1)[1], t in 1:size(x1)[2]]);

In [ ]:
q = [zeros(365) for i in 1:10, t in 1:10];

# rolling sum > 0

In [ ]:
ttop = 0.75;
tbase = 0.25;
agdd = 0.01;

In [ ]:
tavg = rand(10,10,730) + 273.15;

In [ ]:
gdd = tavg - tbase;

In [ ]:
gdd_ts = gdd[2,2,:];

In [ ]:
function calc_agdd_growth(gdd_ts::Array{Float64,1}, ttop::Float64, tbase::Float64, agdd::Float64)
   
    grow_range = (gdd_ts.>(0)) .& (gdd_ts.<(ttop-tbase))
    [sum(cumprod(grow_range[i:(i+364)]) .* gdd_ts[i:(i+364)], 1)[1] > agdd for i in 1:365]

end

In [ ]:
function predict_growth()

## OLDER

In [ ]:
using BenchmarkTools

In [ ]:
const TTOP = 0.75;
const TBASE = 0.25;
const AGDD = 0.01;

In [ ]:
function calc_daily_growth1(x)
    grow_range = (x.>(TBASE)) .& (x.<(TTOP))
    [sum(cumprod(grow_range[i:(i+364)]) .* (x[i:(i+364)] - TBASE), 1)[1] > AGDD for i in 1:365]
end

In [ ]:
@time calc_daily_growth1(rand(730));

In [ ]:
function calc_daily_growth2(x::Array{Float64,1})
    
    #find range of growth
    grow_range = (x.>(TBASE)) .& (x.<(TTOP))

    gdd = zeros(Float64, size(x))
    gdd[grow_range] = x[grow_range] - TBASE #Later, adjust for Kelvin
    
    [sum(cumprod(grow_range[i:(i+364)]) .* gdd[i:(i+364)], 1)[1] > AGDD for i in 1:365]
    
end

In [ ]:
@time calc_daily_growth2(rand(730));

In [ ]:
Profile.print(format=:flat)

In [ ]:
function calc_daily_growth(x)
    
    #find range of growth
    grow_range = (x.>(TBASE)) .& (x.<(TTOP))

    x -= TBASE #Later, adjust for Kelvin
    
    [sum(cumprod(grow_range[i:(i+364)]) .* x[i:(i+364)], 1)[1] > AGDD for i in 1:365]
    
end

In [ ]:
@benchmark calc_daily_growth(rand(730))

In [ ]:
addprocs(4)

In [ ]:
@everywhere function calc_daily_growth(x)
    
    #find range of growth
    grow_range = (x.>(TBASE)) .& (x.<(TTOP))
    
    #[sum(cumprod(grow_range[i:(i+364)]) .* (x[i:(i+364)] - TBASE), 1)[1] > AGDD for i in 1:365]
    gdd = zeros(Float64, size(x))
    gdd[grow_range] = x[grow_range] - TBASE #Later, adjust for Kelvin
    
    [sum(cumprod(grow_range[i:(i+364)]) .* gdd[i:(i+364)], 1)[1] > AGDD for i in 1:365]
    
end

In [ ]:
@everywhere TTOP = 0.75
@everywhere TBASE = 0.25
@everywhere AGDD = 0.01

In [ ]:
x = rand(1440, 720, 730);

In [ ]:
x - TBASE

In [ ]:
@time grow_range = ((x.>(TBASE)) .& (x.<(TTOP)))

In [ ]:
t = x[1,1,:];

In [ ]:
@time w = [x[i,t,:] for i in 1:size(x)[1], t in 1:size(x)[2]];

In [ ]:
x = rand(10,10,730);

In [ ]:
t = Array{Array{Bool,2},2}

In [ ]:
@time x_res1  = pmap(calc_daily_growth, [x[i,t,:] for i in 1:size(x)[1], t in 1:size(x)[2]]);

In [ ]:
size(x_res1)[1]

In [ ]:
function format_results(x)
    
    x_new = zeros(size(x)[1], size(x)[2],365)
    counter = 1
    for i in 1:10, t in 1:10
        x_new[i,t,:] = x[counter]
        counter +=1
    end
    
    return x_new

end

In [ ]:
@time x_new = create_matrix(x_res);

In [ ]:
function format_results(x)
    
    x_new = zeros(size(x)[1], size(x)[2],365)
    counter = 1
    for i in 1:10, t in 1:10
        x_new[i,t,:] = x[counter]
        counter +=1
    end
    
    return x_new

end

In [ ]:
[(i,t) for i in 1:10, t in 1:10]
    

In [ ]:
cat(3, q[:])

In [ ]:
reshape(cat(3, q...),10,10,365)

In [ ]:
cat(3, map(x->x', q)...)

In [ ]:
cat(3, q...)

In [ ]:
cat(3, map(x->x', q)...)

In [ ]:
vcat(map(x->x', q)...)

In [ ]:
cat(3, q)

In [ ]:
x_new = zeros(10,10,365);

In [ ]:
for (t, i)

In [ ]:
for (t,i,z) in 1:10, y in 1:10
    print(t,i,z)
end

In [ ]:
for (t, i) in enumerate(q)
   x_new[t] = q[t][:] 
end

In [ ]:
[i[] for (t,i) in enumerate(q)]

In [ ]:
vcat(map(x->x', q)...)

In [ ]:
q

In [ ]:
cat(2,q)

In [ ]:
@time q = mapslices(calc_daily_growth, x, 3);

In [ ]:
@time map(calc_daily_growth, w);

In [ ]:
x = x[1,1,:];

In [ ]:
#find range of growth
grow_range = (x.>(TBASE)) .& (x.<(TTOP))

gdd = zeros(Float64, size(x))
gdd[grow_range] = x[grow_range] - TBASE #Later, adjust for Kelvin

#[sum(cumprod(grow_range[i:(i+364)]) .* gdd[i:(i+364)], 1)[1] > AGDD for i in 1:365]

# adjust TBASE to kelvin later


# grid time slice

## Test

In [ ]:
using Stats

### all zeros

In [ ]:
x = zeros(730);
x_t = zeros(Int8, size(x));
run_val, run_len = rle(x_t)
cs = cumsum(run_len);
last_idx = findfirst(cs.>365, true)
max_val, max_idx = findmax(run_val[1:last_idx] .* run_len[1:last_idx])
longest_start = 

### all ones

In [ ]:
x = ones(730);
x_t = ones(Int8, size(x));
run_val, run_len = rle(x_t)
cs = cumsum(run_len);
last_idx = findfirst(cs.>365, true)
max_val, max_idx = findmax(run_val[1:last_idx] .* run_len[1:last_idx])

### ones only in part 1, at start

In [ ]:
x = [1,1,1,1,1]
x = vcat(x, zeros(730-size(x)[1]));
x_t = zeros(Int8, size(x));
x_t[x.==1] = 1
run_val, run_len = rle(x_t)
cs = cumsum(run_len);
last_idx = findfirst(cs.>365, true)
max_val, max_idx = findmax(run_val[1:last_idx] .* run_len[1:last_idx])

### ones only in part 1, not at start

In [ ]:
x = [0,0,1,1,1,1,1]
x = vcat(x, zeros(730-size(x)[1]));
x_t = zeros(Int8, size(x));
x_t[x.==1] = 1
run_val, run_len = rle(x_t)
cs = cumsum(run_len);
last_idx = findfirst(cs.>365, true)
max_val, max_idx = findmax(run_val[1:last_idx] .* run_len[1:last_idx])

In [ ]:
(cs[max_idx]-max_val+1) : cs[max_idx] 

### ones overlap part2, at start

In [ ]:
x = vcat(ones(400), zeros(330))
x_t = zeros(Int8, size(x));
x_t[x.==1] = 1
run_val, run_len = rle(x_t)
cs = cumsum(run_len);
last_idx = findfirst(cs.>365, true)
max_val, max_idx = findmax(run_val[1:last_idx] .* run_len[1:last_idx])

In [ ]:
(cs[max_idx]-max_val+1) : (cs[max_idx]-max_val + 365)

In [ ]:
x = vcat(zeros(30), ones(400), zeros(300))
x_t = zeros(Int8, size(x));
x_t[x.==1] = 1
run_val, run_len = rle(x_t)
cs = cumsum(run_len);
last_idx = findfirst(cs.>365, true)
max_val, max_idx = findmax(run_val[1:last_idx] .* run_len[1:last_idx])

In [ ]:
(cs[max_idx]-max_val+1) : (cs[max_idx]-max_val + 365)

In [ ]:
x[(cs[max_idx]-max_val+1): (cs[max_idx]-max_val + 365)]

# Old

In [ ]:
using NetCDF;
using Stats;

In [ ]:
rcps = ["rcp45", "rcp85"];

cmips = ["ACCESS1-0", "CSIRO-Mk3-6-0", "MIROC-ESM", "BCC-CSM1-1", 
    "GFDL-CM3", "MIROC-ESM-CHEM", "BNU-ESM", "GFDL-ESM2G", "MIROC5", "CanESM2", 
    "GFDL-ESM2M", "MPI-ESM-LR", "CCSM4", "INMCM4", "MPI-ESM-MR", "CESM1-BGC", 
    "IPSL-CM5A-LR", "MRI-CGCM3", "CNRM-CM5", "IPSL-CM5A-MR", "NorESM1-M"];

In [ ]:
year = 2006;
tb = 10;
tt = 40;
ag = 1825;

In [ ]:
function calc_temp_avg(cmip, rcp, year)
    
    tasmax = ncread(string("data/test/tasmax_day_BCSD_rcp45_r1i1p1_BNU-ESM_", year, ".nc"), "tasmax")
    tasmin = ncread(string("data/test/tasmin_day_BCSD_rcp45_r1i1p1_BNU-ESM_", year, ".nc"), "tasmin")
  
    tavg = (tasmax + tasmin) / 2
    
    return tavg
    
end

In [ ]:
tavg = calc_temp_avg(cmips[1], rcps[1], year);

In [ ]:
x1 = [0,0,0,0,0,0,0,0,0,0];
x2 = [0,0,0,0,0,0,0,0,0,0];

In [ ]:
x = vcat(x1, x2);

In [ ]:
using Stats

In [ ]:
run_val, run_len = rle(x)

In [ ]:
cs = cumsum(run_len)

In [ ]:
last_idx = findfirst(cs.>=5, true)

In [ ]:
max_val, max_idx = findmax(run_val[1:last_idx] .* run_len[1:last_idx])

In [ ]:
sum(x[max_idx:(max_idx+max_val-1)])

In [ ]:
function calc_max_gdd(x, tbase, ttop, agdd)
   
    x_t = zeros(Int8, size(x))
    x_t[(x.>(273.15+tbase)) .& (x.<(273.15+ttop))] = 1;
    #println(x_t)
    
    run_val, run_len = rle(x_t)
    #println([run_val, run_len])
    max_val, max_idx = findmax(run_val .* run_len)
    #println([max_val, max_idx])
    
    # if all zero, return 0
    if max_val == 0
        return 0
    end
    
    if max_val == size(x)[1]
        if sum(x-273.15) > agdd
            return 1
        else
            return 0
        end
    end
    
    cs = cumsum(run_len)
    
    if max_idx == 1
        best_idx = 1:max_val 
    else
        best_idx = cs[max_idx-1]+1 : cs[max_idx]
    end
    
    if sum(x[best_idx]-273.15) > agdd
        return 1
    else
        return 0
    end
    
end

In [ ]:
tavg

In [ ]:
function process_model(cmip, rcp, year, tbase, ttop, req_agdd)
   
    tavg = calc_temp_avg(cmip, rcp, year);
    [calc_max_gdd(tavg[x,y,:], tbase, ttop, req_agdd) for x in 1:size(tavg)[1], y in 1:size(tavg)[2]]
    
end

In [ ]:
function predict_growth(year, tbase, ttop, req_agdd)
    
    rcps = ["rcp45", "rcp85"];

    cmips = ["ACCESS1-0", "CSIRO-Mk3-6-0", "MIROC-ESM", "BCC-CSM1-1", 
        "GFDL-CM3", "MIROC-ESM-CHEM", "BNU-ESM", "GFDL-ESM2G", "MIROC5", "CanESM2", 
        "GFDL-ESM2M", "MPI-ESM-LR", "CCSM4", "INMCM4", "MPI-ESM-MR", "CESM1-BGC", 
        "IPSL-CM5A-LR", "MRI-CGCM3", "CNRM-CM5", "IPSL-CM5A-MR", "NorESM1-M"];
    
    store_mat = zeros(1440, 720)
    
    for rcp in rcps
    
        println(rcp)
        
        for cmip in cmips
           
            println(string("  ", cmip))
            store_mat += process_model(cmip, rcp, year, tbase, ttop, req_agdd)
            
        end
    
    end
    
    store_mat / 42
    
end

In [ ]:
total_mat = zeroes()

In [ ]:
function process_models()

In [ ]:
@time best_gdd = [calc_max_gdd(tavg[x,y,:], tb, tt, ag) for x in 1:size(tavg)[1], y in 1:size(tavg)[2]]

In [ ]:
maximum(best_gdd)

In [ ]:
using Plots

In [ ]:
spy(best_gdd)

In [ ]:
# set missing (extreme max) 0 NOT NECESSARY, TTOP got it
#tavg[tavg.==maximum(tavg)] = 0; 

In [ ]:
tavg_bin = zeros(Int8, size(tavg))
tavg_bin[tavg.>tbase * tavg.<ttop] = 1;

In [ ]:
# assumes array of only 0 and 1
function longest_run_index_range(x)
   
    run_val, run_len = rle(x)
    max_len, max_idx = findmax(run_val .* run_len)
    
    if max_len == 0
        return 0:0
    end
    
    cs = cumsum(run_len)
    cs[max_idx-1]+1 : cs[max_idx]

    
end

In [ ]:
mapslices(longest_run_index_range, tavg_bin, 3)

In [ ]:
rle(tavg_bin[1,1,:])

In [ ]:
for x in size(tavg)[1]

    for y in size(tavg)[2]
       
        rle()
        
    end
    
end


In [ ]:
# find where greater than tbase and less than tmax
tavg_bin = tavg.>tbase * tavg.<ttop;

In [ ]:
rle(tavg_bin[1,1,:])

In [ ]:
convert(Array{Int64, 1}, tavg_bin)

In [ ]:
using Stats;

In [ ]:
tavg_bin[1,1,:]

- for a given year
- go through each model
- get tavg data
- 